# Generalized Classes For Deep Learning models using Keras

We will build a classes to predict the results.

Here we alter the datasets in a slightly random way to prove that the class is generalized and flexible.  Problems and bugs will be left in the code so engineers studying machine learning have to be active hands-on instead of merely cloning and running.

# Regression Class using Keras

Here we will randomly alter the hourly wages data using pandas.  To learn that on your own try this: https://github.com/Synchronicity89/Minimally-Sufficient-Pandas

In [22]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
import numpy as np

In [23]:
#read in training data
#train_df = pd.read_html("https://github.com/eijaz1/Deep-Learning-in-Keras-Tutorial/blob/master/data/hourly_wages_data.csv")
train_df = pd.read_csv('Datasets/hourly_wages_data.csv')

#alter data structure
#With any library that is too foreign or confusing, create your own wrapper that is similar to a language/framework you are familiar with
#Mine is .NET, yours could be different.  In .NET there are DataSet, DataRow etc
#Then eventually everything you actually use is in that wrapper. Of course copy it to its own py file and make a library and reuse it.
#So there is no need to copy/paste code every time.  Here I will try to use Pandas without making a wrapper
filt = (train_df['education_yrs'] > 0) & (train_df['experience_yrs'] > 0)

train_df.loc[filt, 'wage_per_hour'] = np.log(train_df['education_yrs'] * train_df['experience_yrs'])

print(train_df)
    


     wage_per_hour  union  education_yrs  experience_yrs  age  female  marr  \
0         5.123964      0              8              21   35       1     1   
1         5.934894      0              9              42   57       1     1   
2         2.484907      0             12               1   19       0     0   
3         3.871201      0             12               4   22       0     0   
4         5.318120      0             12              17   35       0     1   
5         4.762174      1             13               9   28       0     0   
6         5.598422      0             10              27   43       0     0   
7         4.682131      0             12               9   27       0     0   
8         5.170484      0             16              11   33       0     1   
9         4.682131      0             12               9   27       0     0   
10        5.318120      1             12              17   35       0     1   
11        5.429346      1             12            

C:\Users\baker\AppData\Local\conda\conda\envs\p35\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


Next, we need to split up our dataset into inputs (train_X) and our target (train_y). Our input will be every column except 'wage_per_hour' because 'wage_per_hour' is what we will be attempting to predict. Therefore, 'wage_per_hour' will be our target.

We will use the pandas 'drop' function to drop the column 'wage_per_hour' from our dataframe and store it in the variable 'train_X'. This will be our input.

In [24]:
#create a dataframe with all training data except the target column
train_X = train_df.drop(columns=['wage_per_hour'])

#check that the target variable has been removed
train_X.head()

,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
0,0,8,21,35,1,1,0,1,0
1,0,9,42,57,1,1,0,1,0
2,0,12,1,19,0,0,0,1,0
3,0,12,4,22,0,0,0,0,0
4,0,12,17,35,0,1,0,0,0


In [25]:
#create a dataframe with only the target column
train_y = train_df[['wage_per_hour']]

#view dataframe
train_y.head()

,wage_per_hour
0,5.123964
1,5.934894
2,2.484907
3,3.871201
4,5.318120


The model type that we will be using is Sequential. Sequential is the easiest way to build a model in Keras. It allows you to build a model layer by layer. Each layer has weights that correspond to the layer the follows it.

We use the 'add()' function to add layers to our model. We will add two layers and an output layer.

'Dense' is the layer type. Dense is a standard layer type that works for most cases. In a dense layer, all nodes in the previous layer connect to the nodes in the current layer.

We have 10 nodes in each of our input layers. This number can also be in the hundreds or thousands. Increasing the number of nodes in each layer increases model capacity. I will go into further detail about the effects of increasing model capacity shortly.

'Activation' is the activation function for the layer. An activation function allows models to take into account nonlinear relationships. For example, if you are predicting diabetes in patients, going from age 10 to 11 is different than going from age 60â??61.

The activation function we will be using is ReLU or Rectified Linear Activation. Although it is two linear pieces, it has been proven to work well in neural networks.

The first layer needs an input shape. The input shape specifies the number of rows and columns in the input. The number of columns in our input is stored in 'n_cols'. There is nothing after the comma which indicates that there can be any amount of rows.

The last layer is the output layer. It only has one node, which is for our prediction.

Next, we need to compile our model. Compiling the model takes two parameters: optimizer and loss.

The optimizer controls the learning rate. We will be using 'Adam' as our optimizer. Adam is generally a good optimizer to use for many cases. The Adam optimizer adjusts the learning rate throughout training.

The learning rate determines how fast the optimal weights for the model are calculated. A smaller learning rate may lead to more accurate weights (up to a certain point), but the time it takes to compute the weights will be longer.

For our loss function, we will use 'mean_squared_error'. It is calculated by taking the average squared difference between the predicted and actual values. It is a popular loss function for regression problems. The closer to 0 this is, the better the model performed.



In [26]:
from keras import backend as K
import tensorflow as tf

gpus = K.tensorflow_backend._get_available_gpus()
processor = ('/gpu:' + str(len(gpus)-1)) if len(gpus) > 0 else '/cpu:0'

with tf.device(processor):
    #create model
    model = Sequential()

    #get number of columns in training data
    n_cols = train_X.shape[1]

    #add model layers
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))

    #compile model using mse as a measure of model performance
    model.compile(optimizer='adam', loss='mean_squared_error')

    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)

Now we will train our model. To train, we will use the 'fit()' function on our model with the following five parameters: training data (train_X), target data (train_y), validation split, the number of epochs and callbacks.

The validation split will randomly split the data into use for training and testing. During training, we will be able to see the validation loss, which gives the mean squared error of our model on the validation set. We will set the validation split at 0.2, which means that 20% of the training data we provide in the model will be set aside for testing model performance.

The number of epochs is the number of times the model will cycle through the data. The more epochs we run, the more the model will improve, up to a certain point. After that point, the model will stop improving during each epoch. In addition, the more epochs, the longer the model will take to run. To monitor this, we will use 'early stopping'.

Early stopping will stop the model from training before the number of epochs is reached if the model stops improving. We will set our early stopping monitor to 3. This means that after 3 epochs in a row in which the model doesn't improve, training will stop. Sometimes, the validation loss can stop improving then improve in the next epoch, but after 3 epochs in which the validation loss doesn't improve, it usually won't improve again.

In [27]:
#train model
model.fit(train_X, train_y, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])

Train on 427 samples, validate on 107 samples
Epoch 1/30
427/427 [==============================] - 0s 1ms/step - loss: 454.5973 - val_loss: 318.9381
Epoch 2/30
427/427 [==============================] - 0s 173us/step - loss: 337.4681 - val_loss: 239.0106
Epoch 3/30
427/427 [==============================] - 0s 166us/step - loss: 251.5616 - val_loss: 183.9263
Epoch 4/30
427/427 [==============================] - 0s 154us/step - loss: 189.5674 - val_loss: 145.4595
Epoch 5/30
427/427 [==============================] - 0s 154us/step - loss: 146.2999 - val_loss: 117.1560
Epoch 6/30
427/427 [==============================] - 0s 157us/step - loss: 113.8245 - val_loss: 95.1978
Epoch 7/30
427/427 [==============================] - 0s 152us/step - loss: 88.2571 - val_loss: 77.9270
Epoch 8/30
427/427 [==============================] - 0s 159us/step - loss: 68.5609 - val_loss: 63.6309
Epoch 9/30
427/427 [==============================] - 0s 164us/step - loss: 52.4182 - val_loss: 51.9927
Epoch 10/

Making predictions on new data

If you want to use this model to make predictions on new data, we would use the 'predict()' function, passing in our new data.
The output would be 'wage_per_hour' predictions.

In [28]:
#example on how to use our newly trained model on how to make predictions on unseen data
#we will pretend our new data is saved in a dataframecalled 'test_X')

test_df = pd.read_csv('Datasets/hourly_wages_data.csv')
test_X = test_df.drop(columns=['wage_per_hour'])

test_y_predictions = model.predict(test_X)

print (test_y_predictions)

[[ 4.58708096e+00]
 [ 8.76239586e+00]
 [ 3.64182770e-01]
 [ 6.68535113e-01]
 [ 3.41246963e+00]
 [ 1.42931485e+00]
 [ 5.57780504e+00]
 [ 1.65348697e+00]
 [ 1.59766364e+00]
 [ 1.65348697e+00]
 [ 3.32847977e+00]
 [ 3.81866622e+00]
 [ 5.78581667e+00]
 [ 6.17308760e+00]
 [ 6.12238741e+00]
 [ 7.47785187e+00]
 [ 9.28798962e+00]
 [ 5.32926941e+00]
 [ 3.23157072e+00]
 [ 6.70726109e+00]
 [ 3.09120631e+00]
 [ 8.89943790e+00]
 [ 1.65348697e+00]
 [ 2.88518047e+00]
 [ 4.66584444e+00]
 [ 9.67216587e+00]
 [ 1.42270529e+00]
 [ 6.23616171e+00]
 [ 1.30457783e+00]
 [ 1.43028009e+00]
 [ 2.34767652e+00]
 [ 9.20116329e+00]
 [ 4.14874506e+00]
 [-8.71518254e-02]
 [ 3.90265632e+00]
 [ 7.24038506e+00]
 [ 4.03427982e+00]
 [ 6.99542999e+00]
 [ 5.57322979e-01]
 [ 1.53541613e+00]
 [-6.01648092e-02]
 [ 2.42274308e+00]
 [ 2.86097431e+00]
 [ 3.50937629e+00]
 [ 1.08189607e+00]
 [ 3.46311116e+00]
 [ 1.83319569e+00]
 [ 2.70428228e+00]
 [ 6.89894962e+00]
 [ 1.64813614e+00]
 [ 1.20484900e+00]
 [ 1.97865951e+00]
 [ 1.1208733

As you increase the number of nodes and layers in a model, the model capacity increases. Increasing model capacity can lead to a more accurate model, up to a certain point, at which the model will stop improving. Generally, the more training data you provide, the larger the model should be. We are only using a tiny amount of data, so our model is pretty small. The larger the model, the more computational capacity it requires and it will take longer to train.

Let's create a new model using the same training data as our previous model. This time, we will add a layer and increase the nodes in each layer to 200. We will train the model to see if increasing the model capacity will improve our validation score.

In [29]:
#training a new model on the same data to show the effect of increasing model capacity

#create model
model_mc = Sequential()

#add model layers
model_mc.add(Dense(200, activation='relu', input_shape=(n_cols,)))
model_mc.add(Dense(200, activation='relu'))
model_mc.add(Dense(200, activation='relu'))
model_mc.add(Dense(1))

#compile model using mse as a measure of model performance
model_mc.compile(optimizer='adam', loss='mean_squared_error')

In [30]:
#train model
model_mc.fit(train_X, train_y, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])

Train on 427 samples, validate on 107 samples
Epoch 1/30
427/427 [==============================] - 0s 1ms/step - loss: 5.8293 - val_loss: 1.5123
Epoch 2/30
427/427 [==============================] - 0s 91us/step - loss: 1.0216 - val_loss: 0.8081
Epoch 3/30
427/427 [==============================] - 0s 108us/step - loss: 0.3349 - val_loss: 0.6554
Epoch 4/30
427/427 [==============================] - 0s 91us/step - loss: 0.2064 - val_loss: 0.6740
Epoch 5/30
427/427 [==============================] - 0s 101us/step - loss: 0.1538 - val_loss: 0.7826
Epoch 6/30
427/427 [==============================] - 0s 101us/step - loss: 0.1436 - val_loss: 0.8272


We can see that by increasing our model capacity, we have improved our validation loss from 29.65 in our old model to 28.39 in our new model.

# Classification Model using Keras

Now let's move on to building our model for classification. Since many steps will be a repeat from the previous model, I will only go over new concepts.

For this next model, we are going to predict if patients have diabetes or not.

In [ ]:
#read in training data
train_df_2 = pd.read_csv('Datasets/diabetes_data.csv')

#view data structure
train_df_2.head()

In [ ]:
#create a dataframe with all training data except the target column
train_X_2 = train_df_2.drop(columns=['diabetes'])

#check that the target variable has been removed
train_X_2.head()

When separating the target column, we need to call the 'to_categorical()' function so that column will be 'one-hot encoded'. Currently, a patient with no diabetes is represented with a 0 in the diabetes column and a patient with diabetes is represented with a 1. 

With one-hot encoding, the integer will be removed and a binary variable is inputted for each category. In our case, we have two categories: no diabetes and diabetes.

A patient with no diabetes will be represented by [1 0] and a patient with diabetes will be represented by [0 1].

In [ ]:
#one-hot encode target column
train_y_2 = to_categorical(train_df_2.diabetes)

#vcheck that target column has been converted
train_y_2[0:5]

The last layer of our model has 2 nodes - one for each option: the patient has diabetes or they don't.

The activation is 'softmax'. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has a higher probability.

We will use 'categorical_crossentropy' for our loss function. This is the most common choice for classification. A lower score indicates that the model is performing better.

To make things even easier to interpret, we will use the 'accuracy' metric to see the accuracy score on the validation set at the end of each epoch.

In [ ]:
with tf.device(processor):
    #create model
    model_2 = Sequential()

    #get number of columns in training data
    n_cols_2 = train_X_2.shape[1]

    #add layers to model
    model_2.add(Dense(250, activation='relu', input_shape=(n_cols_2,)))
    model_2.add(Dense(250, activation='relu'))
    model_2.add(Dense(250, activation='relu'))
    model_2.add(Dense(2, activation='softmax'))

    #compile model using accuracy to measure model performance
    model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#train model
model_2.fit(train_X_2, train_y_2, epochs=30, validation_split=0.2, callbacks=[early_stopping_monitor])